In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
car_dataframe = pd.read_csv("data/car details v4.csv")
data = (car_dataframe.loc[:, ["Fuel Tank Capacity", "Year", "Price"]]).to_numpy().astype('float64')
data_clean = data[~np.isnan(data).any(axis=1)]
x = data[:, :2]
y = data[:, 2:]
print(x)
print(y)

[[  35. 2017.]
 [  42. 2014.]
 [  35. 2011.]
 ...
 [  45. 2013.]
 [  65. 2018.]
 [  nan 2017.]]
[[ 505000.]
 [ 450000.]
 [ 220000.]
 ...
 [ 240000.]
 [4290000.]
 [ 670000.]]


In [22]:
def compute_cost(x, y, w, b):
    m = y.shape[0]
    return 2

def run_grad_decent(x, y, w, b, alpha):
    return w, b

def run_linear_regression(x, y, alpha = .01, iter = 1000):
    j = x.shape[1]
    w = np.empty([j, 1])
    print(w)
    b = 0
    for _ in range(iter):
        w, b = run_grad_decent(x, y, w, b, alpha)
        # print(compute_cost(x, y, w, b))
    return w, b

In [23]:
run_linear_regression(x, y)

[[0.e+000]
 [5.e-324]]


(array([[0.e+000],
        [5.e-324]]),
 0)